In [70]:
from garmin_fit_sdk import Decoder, Stream
import folium

class Position:
    SEMICIRCLE_TO_DEGREE = 180 / (2**31)

    def __init__(self, latitude: float, longitude: float):
        self.latitude = self.semicircles_to_degrees(latitude)
        self.longitude = self.semicircles_to_degrees(longitude)

    @staticmethod
    def semicircles_to_degrees(value: float) -> float:
        return value * Position.SEMICIRCLE_TO_DEGREE

    def __repr__(self):
        return f"Position(latitude={self.latitude}, longitude={self.longitude})"


def find_center(coordinates):
    if not coordinates:
        return None
    x_center = sum(x for x, y in coordinates) / len(coordinates)
    y_center = sum(y for x, y in coordinates) / len(coordinates)
    return (x_center, y_center)


def parse_fit_file(filepath):
    stream = Stream.from_file(filepath)
    decoder = Decoder(stream)
    messages, errors = decoder.read()
    if errors:
        print(f"Errors found while decoding FIT file: {errors}")
    return messages


def extract_coordinates(record_messages):
    coordinates = []
    position_lat = "position_lat"
    position_long = "position_long"
    for message in record_messages:
        if position_lat in message and position_long in message:
            coordinates.append(
                Position(message[position_lat], message[position_long])
            )
    return coordinates


def create_map(coordinates, output_file="map.html"):
    folium_coordinates = [[coord.latitude, coord.longitude] for coord in coordinates]
    map_center = find_center(folium_coordinates)
    start = (coordinates[0].latitude, coordinates[0].longitude)
    end = (coordinates[-1].latitude, coordinates[-1].longitude)
    m = folium.Map(location=map_center, zoom_start=11)
    folium.Marker(
        location=start,
        tooltip="Start",
        popup="Home sweet home",
        icon=folium.Icon(color="green")
    ).add_to(m)
    folium.Marker(
        location=end,
        tooltip="End",
        popup="Home sweet home",
        icon=folium.Icon(color="red")
    ).add_to(m)
    folium.PolyLine(folium_coordinates, tooltip="route").add_to(m)
    m.save(output_file)
    print(f"Map saved to {output_file}")


def main():
    filepath = '../data/13887398069.fit'
    messages = parse_fit_file(filepath)
    record_mesgs = messages["record_mesgs"]
    coordinates = extract_coordinates(record_mesgs)
    create_map(coordinates)


main()


Map saved to map.html
